# Install detectron2

In [2]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-aeyqlyer
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-aeyqlyer
  Resolved https://github.com/facebookresearch/detectron2.git to commit 400a49c1ec11a18dd25aea3910507bc3bcd15794
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.0 MB/s eta 0:00:0000:01
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6384299 sha256=d868580cfe9677d4689c8791ee68a309cbc86eed69006a82890a078da8f57c60
  Stored in directory: /tmp/pip-ephem-wheel-cache-71qxpfs6/wheels/17/d9/40/60db98e485aa

In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
import pkg_resources
print("detectron2:", pkg_resources.get_distribution("detectron2").version)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.5 ; cuda:  cu124
detectron2: 0.6


In [4]:
# Cell 4: Imports & Config (Updated for FPN Model)

import torch
import torch.nn as nn # Added for potential projection layer
import detectron2
import os
import numpy as np
from PIL import Image
import gc
from tqdm.notebook import tqdm
import logging
import json # Added for loading JSON data

# Detectron2 specific imports
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.model_zoo import model_zoo, get_config_file, get_checkpoint_url
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.structures import Boxes, ImageList

# --- Configuration ---\n# Ensure these match the main notebook's Cell 3 values if running separately
KAGGLE_INPUT_DIR = "/kaggle/input"
KAGGLE_WORKING_DIR = "/kaggle/working"
# Where to save the final .pt files containing feature dictionaries
VISUAL_FEATURE_DIR = os.path.join(KAGGLE_WORKING_DIR, "region_visual_features")

MAX_REGIONS = 36         # Max regions per image (for LXMERT)
# *** UPDATED FOR FPN MODEL ***
# Feature dim from R-101-FPN box_head output (before predictor) is usually 1024
DETECTOR_OUTPUT_DIM = 1024
# Target dim should match what LXMERT expects (e.g., 2048 for base)
# We might need projection if TARGET_FEATURE_DIM remains 2048
TARGET_FEATURE_DIM = 2048
# --- End Configuration ---


# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

# Create output directory
os.makedirs(VISUAL_FEATURE_DIR, exist_ok=True)
logger.info(f"Output directory for features: {VISUAL_FEATURE_DIR}")

# --- Helper Function for Image Paths ---
# (Keep the get_image_path function from the previous version here)
def get_image_path(sample_id, dataset_base_path, split_name):
    """
    Constructs the full image path based on dataset conventions.
    Args:
        sample_id (str): The ID of the sample (e.g., 'TR-1', 'TE-10', 'D-TR-5').
        dataset_base_path (str): Base path for the dataset split
                                 (e.g., "/kaggle/input/axiom-dataset/dataset/Anxiety_Data").
        split_name (str): The name of the split ('train', 'test', 'val').
    Returns:
        str or None: The full path to the image file, or None if not found.
    """
    dataset_name = os.path.basename(dataset_base_path) # "Anxiety_Data" or "Depressive_Data"

    if "Anxiety" in dataset_name:
        # Path: .../Anxiety_Data/anxiety_{split_name}_image/{sample_id}.jpg
        image_folder = os.path.join(dataset_base_path, f"anxiety_{split_name}_image")
        # Assume .jpg extension based on screenshots
        image_filename = f"{sample_id}.jpg"
        full_path = os.path.join(image_folder, image_filename)
        if os.path.exists(full_path):
            return full_path
        else:
            # You could try other extensions like .jpeg here if needed
            logger.debug(f"Anxiety image not found at expected path: {full_path}")
            return None

    elif "Depressive" in dataset_name:
        # Path: .../Depressive_Data/Images/depressive_image/{split_name}/{sample_id}.jpeg (or .jpg)
        image_folder = os.path.join(dataset_base_path, "Images", "depressive_image", split_name)
        # Try common extensions based on screenshots and common usage
        for ext in [".jpeg", ".jpg", ".png"]:
             image_filename = f"{sample_id}{ext}"
             full_path = os.path.join(image_folder, image_filename)
             if os.path.exists(full_path):
                  return full_path
        logger.debug(f"Depressive image not found for {sample_id} in {image_folder} with extensions .jpeg, .jpg, .png.")
        return None

    else:
        logger.warning(f"Unrecognized dataset base path structure: {dataset_base_path}")
        return None

# Train on a custom dataset



In [5]:
# Cell 6: Load Detectron2 Model (Using R-101-FPN COCO Model - Cleaned)

import torch.nn as nn # Ensure nn is imported if running cell independently
from detectron2.config import get_cfg
from detectron2.model_zoo import model_zoo, get_config_file, get_checkpoint_url
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
import logging # Ensure logger is available

logger = logging.getLogger(__name__) # Ensure logger is defined

def load_coco_detector(): # Renamed function for clarity
    """Loads the Faster R-CNN R-101-FPN model trained on COCO."""
    cfg = get_cfg()
    try:
        # USE COCO R-101-FPN MODEL CONFIG
        config_file = "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"
        full_config_path = model_zoo.get_config_file(config_file)
        cfg.merge_from_file(full_config_path)
        logger.info(f"Loaded config file: {full_config_path}")

        # Set thresholds
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1 # Set threshold to 0.1
        logger.info(f"Set cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST to {cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST}")
        # cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 1000 # Default is 1000 for FPN
        # cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5 # Default is 0.5 for FPN

        # Load weights for the COCO model
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_file)
        logger.info(f"Attempting to load weights from: {cfg.MODEL.WEIGHTS}")

        # Set device
        cfg.MODEL.DEVICE = device.type # Assumes 'device' is defined globally
        cfg.freeze()

        # Build model
        logger.info(f"Building model...")
        model = build_model(cfg)
        model.eval()
        logger.info(f"Model built successfully.")

        # Load weights into model
        checkpointer = DetectionCheckpointer(model)
        logger.info(f"Attempting checkpointer.load()...")
        checkpointer.load(cfg.MODEL.WEIGHTS) # Load COCO weights
        logger.info("Weights loaded successfully.")

        # Move model to device
        model.to(device)
        logger.info("Detectron2 Faster R-CNN R-101-FPN (COCO) model loaded successfully to device.")
        return model, cfg

    # Keep error handling as before
    except RuntimeError as e:
         if "No file found" in str(e) or "matching config file" in str(e):
              logger.error(f"Could not find config file or weights for {config_file} at expected paths.")
              logger.error("Please ensure Detectron2 is installed correctly and the model zoo path is valid.")
         elif "CUDA out of memory" in str(e):
             logger.error("CUDA out of memory. Try reducing batch size (if applicable) or using a smaller model.")
         else:
              logger.error(f"Runtime error loading Detectron2 model: {e}", exc_info=True)
         return None, None
    except Exception as e:
        logger.error(f"An unexpected error occurred loading the detector: {e}", exc_info=True)
        return None, None

# --- Calling the Updated Function ---
logger.info("Calling load_coco_detector()...")
object_detector_model, detector_cfg = load_coco_detector()
if object_detector_model is not None:
     logger.info("Detector model loaded successfully.")
else:
     logger.error("Detector model failed to load.")


# Define projection layer here. Needed because 1024 (detector) != 2048 (target)
projection_layer = None
if object_detector_model is not None and DETECTOR_OUTPUT_DIM != TARGET_FEATURE_DIM:
    logger.info(f"Defining projection layer {DETECTOR_OUTPUT_DIM} -> {TARGET_FEATURE_DIM}")
    projection_layer = nn.Linear(DETECTOR_OUTPUT_DIM, TARGET_FEATURE_DIM).to(device).eval()
elif object_detector_model is not None:
     logger.info(f"Projection layer not needed (Detector output dim {DETECTOR_OUTPUT_DIM} == Target dim {TARGET_FEATURE_DIM}).")
else:
     logger.info(f"Skipping projection layer definition as model is None.")

model_final_f6e8b1.pkl: 243MB [00:00, 290MB/s]                             


In [6]:
# Cell 7: Feature Extraction Function (Cleaned)

import torch
from detectron2.structures import Boxes, Instances
from PIL import Image
import numpy as np
import logging
import traceback # Keep for unexpected errors

# Make sure logger is available
logger = logging.getLogger(__name__)

def extract_features_from_image_vg(img_path, model, cfg, device, projection_layer,
                                   max_regions=MAX_REGIONS,
                                   target_dim=TARGET_FEATURE_DIM,
                                   detector_output_dim=DETECTOR_OUTPUT_DIM):
    """
    Extracts features and normalized boxes using the loaded model.
    Applies projection if needed, pads/truncates, and creates attention mask.
    """
    try:
        with Image.open(img_path) as img_pil:
            img_pil = img_pil.convert("RGB")
            img_cv2 = np.array(img_pil)[:, :, ::-1].copy()

        height, width = img_cv2.shape[:2]
        if height == 0 or width == 0:
            logger.warning(f"Image has zero dimension: {img_path}. Skipping.")
            return None

        with torch.no_grad():
            image_tensor = torch.as_tensor(img_cv2.astype("float32").transpose(2, 0, 1))
            inputs = [{"image": image_tensor, "height": height, "width": width}]
            images = model.preprocess_image(inputs)
            features = model.backbone(images.tensor)
            proposals, _ = model.proposal_generator(images, features)
            proposals = proposals[0]
            features_list = [features[f] for f in model.roi_heads.box_in_features]

            # pred_outputs is likely a list containing ONE Instances object
            pred_outputs, _ = model.roi_heads(images, features, [proposals])

            # Validation and Access
            if not pred_outputs or not isinstance(pred_outputs, list) or len(pred_outputs) == 0:
                logger.warning(f"pred_outputs list is empty or not a list for {img_path}. Skipping.")
                return None
            if not isinstance(pred_outputs[0], Instances):
                 logger.warning(f"First element of pred_outputs is not an Instances object (Type: {type(pred_outputs[0])}) for {img_path}. Skipping.")
                 return None

            instances_gpu = pred_outputs[0] # Access the Instances object directly

            # Check length BEFORE further processing or .to(cpu)
            if len(instances_gpu) == 0:
                logger.debug(f"Instances object has length 0 (no detections after NMS) for {img_path}. Skipping.")
                return None # Exit cleanly if no instances remain

            # If len > 0, proceed to move to CPU and extract data
            try:
                instances = instances_gpu.to("cpu")
            except Exception as cpu_err:
                logger.error(f"Error during .to('cpu'): {cpu_err} for {img_path}", exc_info=True)
                return None

            # Filter and Select Top Regions
            if not instances.has("scores") or not instances.has("pred_boxes"):
                 logger.warning(f"Instances missing 'scores' or 'pred_boxes' after to(cpu) for {img_path}. Skipping.")
                 return None

            scores = instances.scores
            pred_boxes = instances.pred_boxes.tensor

            keep_indices = torch.where(scores >= cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)[0]
            num_kept_after_thresh = len(keep_indices)

            if num_kept_after_thresh == 0:
                 logger.debug(f"No instances passed score threshold {cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST}. Max score: {scores.max():.3f}. Image: {img_path}. Skipping.")
                 return None

            num_to_select = min(num_kept_after_thresh, max_regions)
            scores_above_thresh = scores[keep_indices]
            _, top_k_relative_indices = torch.topk(scores_above_thresh, num_to_select, sorted=False)
            final_indices = keep_indices[top_k_relative_indices]

            # logger.info(f"Kept {num_to_select} instances for {img_path} (Max score: {scores[final_indices].max():.3f})") # Optional: log success details
            final_boxes_topk = pred_boxes[final_indices]

            # --- Re-pooling, Projection, Normalization, Padding ---
            final_box_list_gpu = [Boxes(final_boxes_topk).to(device)]
            final_features_pooled = model.roi_heads.box_pooler(features_list, final_box_list_gpu)
            final_features_unprojected = model.roi_heads.box_head(final_features_pooled)

            if projection_layer is not None:
                projected_features = projection_layer(final_features_unprojected)
            else:
                projected_features = final_features_unprojected

            whwh = torch.tensor([width, height, width, height], device=device, dtype=torch.float32)
            normalized_boxes = final_boxes_topk.to(device) / whwh
            normalized_boxes = torch.clamp(normalized_boxes, 0.0, 1.0)

            num_final_regions = projected_features.shape[0]
            padded_features = torch.zeros((max_regions, target_dim), dtype=projected_features.dtype, device=device)
            padded_boxes = torch.zeros((max_regions, 4), dtype=normalized_boxes.dtype, device=device)
            vis_mask = torch.zeros(max_regions, dtype=torch.long, device=device)

            padded_features[:num_final_regions] = projected_features
            padded_boxes[:num_final_regions] = normalized_boxes
            vis_mask[:num_final_regions] = 1

            # logger.debug(f"Successfully processed {img_path}") # Optional: Log success
            return {
                "features": padded_features.cpu(),
                "boxes": padded_boxes.cpu(),
                "visual_attention_mask": vis_mask.cpu()
            }

    except FileNotFoundError:
        logger.error(f"Image file not found: {img_path}")
        return None
    except Exception as e:
        # Log the full traceback for unexpected errors
        logger.error(f"Unexpected error processing image {img_path}: {e}", exc_info=True)
        # traceback.print_exc() # Optionally print traceback directly if logger isn't detailed enough
        return None

In [7]:
# Cell 8: Main Feature Extraction Loop (Cleaned)

import os
import json
import torch
import gc
from tqdm.notebook import tqdm
import logging

# Make sure logger is available
logger = logging.getLogger(__name__)

# Make sure helper function is defined (or imported if moved)
# def get_image_path(...): ...

def process_dataset_split(dataset_name, split_name, json_path, dataset_base_path, model, cfg, device, projection_layer, output_file, max_regions=MAX_REGIONS, target_dim=TARGET_FEATURE_DIM):
    """Processes all images for a given JSON split and saves features."""
    logger.info(f"Starting processing for: {dataset_name} - {split_name}")
    logger.info(f"JSON source: {json_path}")
    logger.info(f"Output file: {output_file}")

    if model is None:
        logger.error("Object detector model not loaded. Cannot process.")
        return

    all_features_map = {}  # Dictionary to store features: {sample_id: feature_dict}

    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        logger.info(f"Loaded {len(data)} samples from {json_path}")
    except FileNotFoundError:
        logger.error(f"JSON file not found: {json_path}. Skipping this split.")
        return
    except json.JSONDecodeError:
        logger.error(f"Error decoding JSON file: {json_path}. Skipping this split.")
        return
    except Exception as e:
        logger.error(f"Failed to load or parse JSON {json_path}: {e}. Skipping this split.", exc_info=True)
        return

    processed_count = 0
    skipped_count = 0
    error_count = 0

    # Iterate through samples listed in the JSON
    for sample in tqdm(data, desc=f"Extracting {split_name} features ({dataset_name})"):
        sample_id = sample.get('sample_id', sample.get('id', sample.get('image_id')))
        if not sample_id:
            logger.warning(f"Sample ID not found in record: {sample}. Skipping.")
            skipped_count += 1
            continue
        sample_id = str(sample_id)

        # Construct image path using the helper function
        image_path = get_image_path(sample_id, dataset_base_path, split_name) # Assumes get_image_path is defined

        if not image_path:
            logger.warning(f"Image path not found for sample_id: {sample_id} (split: {split_name}). Skipping.")
            skipped_count += 1
            continue

        # Extract features for this image
        visual_data = extract_features_from_image_vg(
            img_path=image_path,
            model=object_detector_model, # Use globally defined model
            cfg=detector_cfg,           # Use globally defined cfg
            device=device,              # Use globally defined device
            projection_layer=projection_layer, # Use globally defined projection_layer
            max_regions=max_regions,
            target_dim=target_dim,
            detector_output_dim=DETECTOR_OUTPUT_DIM # Use global DETECTOR_OUTPUT_DIM
        )

        if visual_data:
            all_features_map[sample_id] = visual_data
            processed_count += 1
        else:
            # Feature extraction returned None (error/skip already logged inside the function)
            error_count += 1

    logger.info(f"Finished processing {split_name} for {dataset_name}. "
                f"Successfully processed: {processed_count}, Skipped (no ID/image): {skipped_count}, Errors/No Detections: {error_count}")

    # Save the collected features
    if all_features_map:
        logger.info(f"Saving {len(all_features_map)} extracted region features to {output_file}...")
        try:
            torch.save(all_features_map, output_file)
            logger.info(f"Region features saved successfully to {output_file}")
        except Exception as e:
            logger.error(f"Failed to save features to {output_file}: {e}", exc_info=True)
    else:
        logger.warning(f"No features were successfully extracted for {dataset_name} - {split_name} (or all resulted in errors/no detections). Output file not saved.")


# --- Define Paths and Run Extraction Loop ---

# Ensure global variables like object_detector_model, detector_cfg, device,
# KAGGLE_INPUT_DIR, VISUAL_FEATURE_DIR, etc., are defined from previous cells.
if object_detector_model is not None:
    logger.info("Model loaded. Proceeding with feature extraction loop.")

    # Use KAGGLE_INPUT_DIR variable defined in Cell 4 (should be /kaggle/input)
    anxiety_base_dir = os.path.join(KAGGLE_INPUT_DIR, "dataset", "Anxiety_Data")
    depression_base_dir = os.path.join(KAGGLE_INPUT_DIR, "dataset", "Depressive_Data")
    output_feature_dir = VISUAL_FEATURE_DIR

    # Verify paths exist (optional check)
    if not os.path.exists(anxiety_base_dir): logger.warning(f"Anxiety base directory not found: {anxiety_base_dir}")
    if not os.path.exists(depression_base_dir): logger.warning(f"Depression base directory not found: {depression_base_dir}")
    if not os.path.exists(output_feature_dir): logger.info(f"Creating output directory: {output_feature_dir}"); os.makedirs(output_feature_dir)


    datasets_to_process = [
        ("anxiety", "train", os.path.join(anxiety_base_dir, "anxiety_train.json"), anxiety_base_dir, os.path.join(output_feature_dir, "anxiety_train_region_features.pt")),
        ("anxiety", "test", os.path.join(anxiety_base_dir, "anxiety_test.json"), anxiety_base_dir, os.path.join(output_feature_dir, "anxiety_test_region_features.pt")),
        ("depression", "train", os.path.join(depression_base_dir, "train.json"), depression_base_dir, os.path.join(output_feature_dir, "depression_train_region_features.pt")),
        ("depression", "test", os.path.join(depression_base_dir, "test.json"), depression_base_dir, os.path.join(output_feature_dir, "depression_test_region_features.pt")),
        ("depression", "val", os.path.join(depression_base_dir, "val.json"), depression_base_dir, os.path.join(output_feature_dir, "depression_val_region_features.pt"))
    ]
    logger.info(f"List of datasets/splits to process created (length {len(datasets_to_process)}).")


    # Process each defined dataset split
    logger.info("Starting the main loop over datasets_to_process...")
    for name, split, json_p, data_base_p, out_f in datasets_to_process:
        logger.info(f"--- Checking prerequisites for loop iteration: {name} - {split} ---")
        logger.info(f"JSON path: {json_p} | Output file path: {out_f}")

        # Check if the corresponding JSON exists before processing
        if os.path.exists(json_p):
            # Check if the output file already exists
            if not os.path.exists(out_f):
                logger.info(f"Output file not found. Calling process_dataset_split for {name} - {split}.")
                process_dataset_split(
                    dataset_name=name,
                    split_name=split,
                    json_path=json_p,
                    dataset_base_path=data_base_p,
                    model=object_detector_model,
                    cfg=detector_cfg,
                    device=device,
                    projection_layer=projection_layer,
                    output_file=out_f,
                    max_regions=MAX_REGIONS,
                    target_dim=TARGET_FEATURE_DIM
                )
                # Clean up GPU memory after processing each split
                logger.info(f"Cleaning up memory after {name} - {split}...")
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            else:
                logger.info(f"Output file exists: {out_f}. Skipping extraction for {name} - {split}.")
        else:
            logger.warning(f"JSON file not found: {json_p}. Skipping iteration for {name} - {split}.")
        logger.info(f"--- Finished checks/processing for loop iteration: {name} - {split} ---\n")

    logger.info("Region feature extraction process finished for all defined splits.")
    # Optional cleanup
    # del object_detector_model, detector_cfg, projection_layer
    # gc.collect()
    # if torch.cuda.is_available(): torch.cuda.empty_cache()
    # logger.info("Detector model and config cleaned up from memory.")

else:
    logger.error("Object detection model was not loaded successfully in prior cells. CANNOT extract region features.")

logger.info("Cell 8 Execution Reached End.")

Extracting train features (anxiety):   0%|          | 0/2608 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Extracting test features (anxiety):   0%|          | 0/652 [00:00<?, ?it/s]

Extracting train features (depression):   0%|          | 0/8814 [00:00<?, ?it/s]

Extracting test features (depression):   0%|          | 0/520 [00:00<?, ?it/s]

Extracting val features (depression):   0%|          | 0/361 [00:00<?, ?it/s]

In [8]:
# Cell 9: Create Zip Archive of Features

import os

output_dir = "/kaggle/working/region_visual_features"
zip_file_path = "/kaggle/working/axiom_region_features.zip" # Output zip file name

# Check if the directory exists and has files
if os.path.exists(output_dir) and len(os.listdir(output_dir)) > 0:
    print(f"Found output directory: {output_dir}")
    print("Files inside:")
    !ls -lh {output_dir}

    print(f"\nZipping directory {output_dir} into {zip_file_path}...")
    # Use shell zip command: -r for recursive, -q for quiet (less verbose output)
    !zip -r -q {zip_file_path} {output_dir}

    # Verify zip file creation
    if os.path.exists(zip_file_path):
        print("\nZip file created successfully:")
        !ls -lh {zip_file_path}
        print(f"\n--> You can now download '{os.path.basename(zip_file_path)}' from the 'Output' section in the Kaggle UI panel.")
    else:
        print("\nError: Zip file creation failed.")
else:
    print(f"Error: Output directory {output_dir} not found or is empty. Cannot create zip file.")
    print("Please ensure the feature extraction in Cell 8 completed and generated .pt files.")

Found output directory: /kaggle/working/region_visual_features
Files inside:
total 3.2G
-rw-r--r-- 1 root root 175M Apr 14 14:10 anxiety_test_region_features.pt
-rw-r--r-- 1 root root 701M Apr 14 14:08 anxiety_train_region_features.pt
-rw-r--r-- 1 root root 137M Apr 14 14:25 depression_test_region_features.pt
-rw-r--r-- 1 root root 2.1G Apr 14 14:24 depression_train_region_features.pt
-rw-r--r-- 1 root root  86M Apr 14 14:26 depression_val_region_features.pt

Zipping directory /kaggle/working/region_visual_features into /kaggle/working/axiom_region_features.zip...

Zip file created successfully:
-rw-r--r-- 1 root root 546M Apr 14 14:27 /kaggle/working/axiom_region_features.zip

--> You can now download 'axiom_region_features.zip' from the 'Output' section in the Kaggle UI panel.
